In [1]:
from games_setup import *
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule, MoleculeStoichiometry
from SBMLLint.common.reaction import Reaction
from SBMLLint.games.som import SOM
from SBMLLint.common.simple_sbml import SimpleSBML

import collections
import itertools
import networkx as nx
import numpy as np
import pandas as pd
#
from SBMLLint.common.stoichiometry_matrix import StoichiometryMatrix
from SBMLLint.games.mesgraph import MESGraph
from SBMLLint.games.games_pp import GAMES_PP, SOMStoichiometry, SOMReaction, TOLERANCE
from SBMLLint.games.games_report import GAMESReport, SimplifiedReaction

ReactionOperation = collections.namedtuple("ReactionOperation", 
    "reaction operation")

Current Directory: /Users/woosubshin/Desktop/ModelEngineering/SBMLLint/notebook


In [2]:
# Model 248 is for canceling error
# Model 14/167 for a type three error / echelon
# Model 7 for type 1 error
# Model 18 for type 2 error

######## examine model 628 for large echelon error ###########
simple = load_file_from_curated_data(51)
# for r in simple.reactions:
#   if r.category != cn.REACTION_BOUNDARY and r.category != cn.REACTION_1_1:
#     print(r.makeIdentifier(is_include_kinetics=False))
m = GAMES_PP(simple)
m.analyze()
gr = GAMESReport(m)

We just analyzed the data...
Type I error:  []
Type II error:  []
Canceling error:  []
Echelon error:  [vTKB: 0.50 {cf6p, cfdp, cg1p, cg6p, cpg, crib5p, cribu5p, cxyl5p} -> ]
Type III error:  []
Type I-SOM error:  set()


In [5]:
print("L^-1", m.lower_inverse.shape)
#print("RREF_DF", m.rref_df.shape)

L^-1 (5, 5)


In [18]:
type1_report, error_num = gr.reportTypeOneError(m.type_one_errors, explain_details=True)
print(type1_report)
print(error_num)


[]


In [19]:
type2_report, error_num = gr.reportTypeTwoError(m.type_two_errors, explain_details=True)
print(type2_report)
print(error_num)


[]


In [33]:
canceling_report, error_num = gr.reportCancelingError(m.canceling_errors, explain_details=True)
print(canceling_report)
print(error_num)


[]


In [7]:
gr.getResultingSeries(m.echelon_errors[0].label)

{ce4p}                                                    0.0
{cglcex}                                                  0.0
{csed7p}                                                  0.0
{cdhap, cgap, cpep, cpg2, cpg3, cpgp, cpyr}               0.0
{cf6p, cfdp, cg1p, cg6p, cpg, crib5p, cribu5p, cxyl5p}   -0.5
Name: vTKB, dtype: float64

In [4]:
m.echelon_errors[0]

vTKB: 0.50 {cf6p, cfdp, cg1p, cg6p, cpg, crib5p, cribu5p, cxyl5p} -> 

In [3]:
echelon_report, error_num = gr.reportEchelonError(m.echelon_errors, explain_details=True)
print(echelon_report)
print(error_num)


We detected a mass imbalance
: 0.50 crib5p -> 

from the following reactions.

1. vTA: cgap + csed7p -> cf6p + ce4p
2. vTKA: crib5p + cxyl5p -> cgap + csed7p
3. vALDO: cfdp -> cdhap + cgap
4. vTKB: ce4p + cxyl5p -> cgap + cf6p

----------------------------------------------------------------------

These uni-uni reactions created mass-equivalence.
5. vrpGluMu: cpg3 -> cpg2
6. vGAPDH: cgap -> cpgp
7. vENO: cpg2 -> cpep
8. vPK: cpep -> cpyr
9. vTIS: cdhap -> cgap
10. vPGK: cpgp -> cpg3

----------------------------------------------------------------------

Based on the reactions above, we have mass-equivalent pseudo reactions.
(pseudo 1.) vTA: {csed7p} + {cdhap, cgap, cpep, cpg2, cpg3, cpgp, cpyr} -> {ce4p} + {cf6p, cfdp, cg1p, cg6p, cpg, crib5p, cribu5p, cxyl5p}
(pseudo 2.) vTKA: 2.00 {cf6p, cfdp, cg1p, cg6p, cpg, crib5p, cribu5p, cxyl5p} -> {csed7p} + {cdhap, cgap, cpep, cpg2, cpg3, cpgp, cpyr}
(pseudo 3.) vALDO: {cf6p, cfdp, cg1p, cg6p, cpg, crib5p, cribu5p, cxyl5p} -> 2.00 {cdhap, 

In [8]:
type3_report, error_num = gr.reportTypeThreeError(m.type_three_errors, explain_details=True)
print(type3_report)
print(error_num)


[]


In [17]:
a = ['1', '2', '3']
type(a[0]) == str

True

In [16]:

# print(m.getNode(simple.getMolecule("CamR_Ca3_ABD_CaMKII")))
# reaction = m.echelon_errors[0]
# print(simple.getReaction("reaction_369"))
operation_df = gr.getOperationMatrix()
operation_df

reaction_label = m.echelon_errors[0].label
operation_series = operation_df.T[reaction_label]
result_series = gr.getResultingSeries(reaction_label)
#operation_df = self.getOperationMatrix()
reaction_operations = gr.convertOperationSeriesToReactionOperations(operation_series)
op_stoi_mat = gr.getOperationStoichiometryMatrix(reaction_operations)
print(op_stoi_mat.index)
op_stoi_mat

Index(['cgap', 'ce4p', 'cgap', 'cgap', 'csed7p', 'crib5p', 'csed7p', 'cfdp',
       'ce4p', 'cxyl5p', 'cf6p', 'cf6p', 'cdhap', 'cxyl5p', 'cgap'],
      dtype='object')


,vTA,vTKA,vALDO,vTKB
cgap,-1.0,1.0,1.0,1.0
ce4p,1.0,0.0,0.0,-1.0
cgap,-1.0,1.0,1.0,1.0
cgap,-1.0,1.0,1.0,1.0
csed7p,-1.0,1.0,0.0,0.0
crib5p,0.0,-1.0,0.0,0.0
csed7p,-1.0,1.0,0.0,0.0
cfdp,0.0,0.0,-1.0,0.0
ce4p,1.0,0.0,0.0,-1.0
cxyl5p,0.0,-1.0,0.0,-1.0


In [55]:
reaction_index = [op.reaction for op in reaction_operations]
fake_series = pd.Series([val.operation for val in reaction_operations], index=reaction_index)
fake_series

reaction_369    1.000000e+00
reaction_175   -2.000000e+00
reaction_216   -6.004800e+15
reaction_124    6.004800e+15
reaction_176    1.000000e+00
reaction_210    6.004800e+15
reaction_27    -6.004800e+15
reaction_293    6.004800e+15
reaction_4      1.000000e+00
reaction_187    1.000000e+00
reaction_112    6.004800e+15
reaction_109   -6.004800e+15
reaction_314    6.004800e+15
reaction_292   -6.004800e+15
reaction_227   -1.000000e+00
reaction_60    -6.004800e+15
dtype: float64

In [37]:
print(m.convertReactionToSOMReaction(simple.getReaction("reaction_216")))
print(m.convertReactionToSOMReaction(simple.getReaction("reaction_124")))

reaction_216: {CamR_Ca2_BD_PP2B} -> {PP2B} + {CamR_Ca2_BD, CamT_Ca2_BD}
reaction_124: {CamR_Ca4_ABCD, CamT_Ca4_ABCD} -> {Ca} + {CamR_Ca3_ABD, CamT_Ca3_ABD}


In [35]:
operation_series[abs(operation_series) > 0]

reaction_369    1.000000e+00
reaction_175   -2.000000e+00
reaction_216   -6.004800e+15
reaction_124    6.004800e+15
reaction_176    1.000000e+00
reaction_210    6.004800e+15
reaction_27    -6.004800e+15
reaction_293    6.004800e+15
reaction_4      1.000000e+00
reaction_187    1.000000e+00
reaction_112    6.004800e+15
reaction_109   -6.004800e+15
reaction_314    6.004800e+15
reaction_292   -6.004800e+15
reaction_227   -1.000000e+00
reaction_60    -6.004800e+15
Name: reaction_369, dtype: float64

In [49]:
echelon_report, error_num = gr.reportEchelonError([m.echelon_errors[0]])
print(echelon_report)
detailed_echelon_report, error_num = gr.reportEchelonError([m.echelon_errors[0]], explain_details=True)
print(detailed_echelon_report)
print(error_num)


We detected a mass imbalance
: 0.50 cxyl5p -> 0.50 cgap + 0.50 cgap + 0.50 cgap

from the following reactions.

1. vTA: cgap + csed7p -> cf6p + ce4p
2. vTKA: crib5p + cxyl5p -> cgap + csed7p
3. vALDO: cfdp -> cdhap + cgap
4. vTKB: ce4p + cxyl5p -> cgap + cf6p

**********************************************************************




UnboundLocalError: local variable 'one_side' referenced before assignment

In [23]:
type3_report, error_num = gr.reportTypeThreeError(m.type_three_errors)
print(type3_report)
detailed_type3_report, error_num = gr.reportTypeThreeError(m.type_three_errors, explain_details=True)
print(detailed_type3_report)
print(error_num)



[]


In [ ]:
"%.2f" % (3.00)

In [41]:
# this will test which model has which error;;
count = 0
for i in range(1, 704):
  try:
    simple = load_file_from_curated_data(i)
    m = GAMES_PP(simple)
    m.analyze(error_details=False)
    if m.echelon_errors:
      print("model", i, " has echelon error!")
      count += 1
      if count == 5:
        break
  except:
    pass

model 49  has echelon error!
model 51  has echelon error!
model 52  has echelon error!
model 105  has echelon error!
model 112  has echelon error!


In [47]:
m.type_one_errors

[]

In [4]:
gr = GAMESReport(m)
print(gr.reportTypeOneError(m.type_one_errors))


FH4 = CH2FH4 by reaction(s):
SHMTr: CH2FH4 -> FH4

However, FH4 < CH2FH4 by reaction(s):
CH2FH4toHCHO: CH2FH4 -> FH4 + HCHO
SHMT: FH4 + serine -> CH2FH4
HCHOtoCH2FH4: FH4 + HCHO -> CH2FH4
**************************************************

CH2FH4 = FH4 by reaction(s):
SHMTr: CH2FH4 -> FH4

FH4 = FH2f by reaction(s):
DHFReductase: FH2f -> FH4

However, CH2FH4 < FH2f by reaction(s):
TYMS: CH2FH4 + dUMP -> FH2f
**************************************************

FFH2 = FH2f by reaction(s):
FFH2syn: FH2f -> FFH2

However, FFH2 < FH2f by reaction(s):
ATIC12: FFH2 + AICAR -> FH2f
**************************************************

MTX1 = MTX1b by reaction(s):
MTX1deg: MTX1b -> MTX1

However, MTX1 < MTX1b by reaction(s):
MTX1off: MTX1b -> MTX1 + DHFRf
MTX1on: MTX1 + DHFRf -> MTX1b
**************************************************

MTX2 = MTX2b by reaction(s):
MTX2deg: MTX2b -> MTX2

However, MTX2 < MTX2b by reaction(s):
MTX2off: MTX2b -> MTX2 + DHFRf
MTX2on: MTX2 + DHFRf -> MTX2b
**********

In [5]:
m.type_two_errors

[[{CHOFH4}, {CH2FH4, FFH2, FH2f, FH4}], [{CH3FH4}, {CH2FH4, FFH2, FH2f, FH4}]]

In [6]:
type_two_report = gr.reportTypeTwoError(m.type_two_errors)
print(type_two_report)


CHOFH4 < FH4 by
ATIC7: CHOFH4 + AICAR -> FH4

FH4 < CHOFH4 by
HCOOHtoCHOFH4: FH4 + formate + ATP -> CHOFH4

CH2FH4 < CHOFH4 by
MTHFD: CH2FH4 + NADP -> CHOFH4

FH4 = CH2FH4 by
SHMTr: CH2FH4 -> FH4
**************************************************


CH3FH4 < FH4 by
MTR: CH3FH4 + homocysteine -> FH4

CH2FH4 < CH3FH4 by
MTHFR: CH2FH4 + NADPH -> CH3FH4

FH4 = CH2FH4 by
SHMTr: CH2FH4 -> FH4
**************************************************


--------------------------------------------------



In [20]:
len(simple.reactions)

587

In [19]:
echelon_report, error_num = gr.reportEchelonError(m.echelon_errors)
#print(echelon_report)
print(error_num)

[28, 67, 104, 139, 166, 187, 206, 229, 248, 270, 302, 336, 362, 388, 416, 439, 465, 498, 532, 565, 599, 633, 665, 702, 738, 777, 814, 848, 877, 898, 923]


In [9]:
error_cycle = []
cycle = m.type_two_errors[0]
cycle2 = cycle[1:] + [cycle[0]]
for first, second in zip(cycle, cycle2):
  print("pairs:")
  print(first, second)
  som1_moles = {m.name for m in list(first.molecules)}
  som2_moles = {m.name for m in list(second.molecules)}
  reaction_data = m.get_edge_data(first, second)[cn.REACTION]
  nodes1 = []
  nodes2 = []
  reaction_labels = []  
  for r in reaction_data:
    reaction = simple.getReaction(r)
    print(reaction.makeIdentifier(is_include_kinetics=False))  
    if reaction.category == cn.REACTION_n_1:
      sources = {r.molecule.name for r in reaction.reactants}
      destinations = {p.molecule.name for p in reaction.products}
    elif reaction.category == cn.REACTION_1_n:
      sources = {p.molecule.name for p in reaction.products}
      destinations = {r.molecule.name for r in reaction.reactants}
    node2 = list(destinations.intersection(som2_moles))[0]
    for node1 in list(sources.intersection(som1_moles)):
      nodes1.append(node1)
      nodes2.append(node2)
      reaction_labels.append(reaction.label)
  error_cycle.append(cn.PathComponents(node1=nodes1, 
                                     node2=nodes2,
                                     reactions=reaction_labels))
error_cycle

pairs:
{CHOFH4} {CH2FH4, FFH2, FH2f, FH4}
ATIC7: CHOFH4 + AICAR -> FH4
pairs:
{CH2FH4, FFH2, FH2f, FH4} {CHOFH4}
HCOOHtoCHOFH4: FH4 + formate + ATP -> CHOFH4
MTHFD: CH2FH4 + NADP -> CHOFH4


[PathComponents(node1=['CHOFH4'], node2=['FH4'], reactions=['ATIC7']),
 PathComponents(node1=['FH4', 'CH2FH4'], node2=['CHOFH4', 'CHOFH4'], reactions=['HCOOHtoCHOFH4', 'MTHFD'])]

In [13]:
  def getSOMPath(som, molecule1, molecule2):
    """
    Create an undirected graph between
    two molecules within a SOM
    and find the shortest path
    :param SOM som:
    :param str mole1:
    :param str mole2:
    :return list-PathComponents som_path:
    """   
#     molecule1 = mole1.name
#     molecule2 = mole2.name
    # construct undirected graph
    subg = nx.Graph()
    # here, every reaction is 1-1 reaction
    for reaction in list(som.reactions):
      node1 = reaction.reactants[0].molecule.name
      node2 = reaction.products[0].molecule.name
      if subg.has_edge(node1, node2):
        reaction_label = subg.get_edge_data(node1, node2)[cn.REACTION]
        # if reaction.label is not already included in the attribute,
        if reaction.label not in set(reaction_label):
          reaction_label = reaction_label + [reaction.label]
      else:
        reaction_label = [reaction.label]    
      subg.add_edge(node1, node2, reaction=reaction_label)
    path = [short_p for short_p in nx.shortest_path(subg, 
                                                    source=molecule1, 
                                                    target=molecule2)]
    som_path = []
    for idx in range(len(path)-1):
      edge_reactions = subg.get_edge_data(path[idx], path[idx+1])[cn.REACTION]
      som_path.append(cn.PathComponents(node1=path[idx], 
                                     node2=path[idx+1],
                                     reactions=edge_reactions))
    return som_path

In [15]:
error_cycle

[PathComponents(node1=['CHOFH4'], node2=['FH4'], reactions=['ATIC7']),
 PathComponents(node1=['FH4', 'CH2FH4'], node2=['CHOFH4', 'CHOFH4'], reactions=['HCOOHtoCHOFH4', 'MTHFD'])]

In [26]:
set(error_cycle[0].node2).intersection(set(error_cycle[1].node1))

{'FH4'}

In [32]:
last_node = None
for one_path in error_cycle:
  comb = zip(one_path.node1, one_path.node2, one_path.reactions)
  if last_node is not None:
    common_molecules = last_node.intersection(set(one_path.node1))
    if common_molecules:
      print("**Already havce common molecules", common_molecules)
    else:
      print("**need to find path between: ", last_node, one_path.node1)
  last_node = set(one_path.node2)
  for tup in comb:
    print(tup[0], cn.LESSTHAN, tup[1], "by", tup[2])
    print(simple.getReaction(tup[2]).makeIdentifier(is_include_kinetics=False))
    print("")
  if len(one_path.node1) > 1:
    for node1, node2 in zip(one_path.node1, one_path.node1[1:]):
      print("node pairs whose equality needs to be examined")
      print(node1, node2)
      mole1 = simple.getMolecule(node1)
      mole2 = simple.getMolecule(node2)
      som = m.getNode(mole1)
      som_path = getSOMPath(som, node1, node2)
      print("som_path is: ", )
      print(som_path)

CHOFH4 < FH4 by ATIC7
ATIC7: CHOFH4 + AICAR -> FH4

**Already havce common molecules {'FH4'}
FH4 < CHOFH4 by HCOOHtoCHOFH4
HCOOHtoCHOFH4: FH4 + formate + ATP -> CHOFH4

CH2FH4 < CHOFH4 by MTHFD
MTHFD: CH2FH4 + NADP -> CHOFH4

node pairs whose equality needs to be examined
FH4 CH2FH4
som_path is: 
[PathComponents(node1='FH4', node2='CH2FH4', reactions=['SHMTr'])]


In [77]:
list(zip(error_cycle[0].node1, error_cycle[0].node2, error_cycle[0].reactions))

[('FH4', 'CHOFH4', 'HCOOHtoCHOFH4'), ('CH2FH4', 'CHOFH4', 'MTHFD')]

In [41]:
cycle[1:] + [cycle[0]]

[{CH2FH4, FFH2, FH2f, FH4}, {CHOFH4}]

In [9]:
gr.mesgraph.som_stoichiometry_matrix.index

Index(['{homocysteine}', '{formate}', '{NADPH}', '{GAR}', '{ATP}', '{CH3FH4}',
       '{DHFRf}', '{HCHO}', '{NADP}', '{dUMP}', '{serine}', '{CHOFH4}',
       '{CH2FH4, FFH2, FH2f, FH4}', '{AICAR, FGAR}',
       '{EMTX, MTX1, MTX1b, MTX2, MTX2b, MTX3, MTX3b, MTX4, MTX4b, MTX5, MTX5b}'],
      dtype='object')

In [7]:
print(m.convertReactionToSOMReaction(simple.getReaction("HCOOHtoCHOFH4")))
print(m.convertReactionToSOMReaction(simple.getReaction("ATIC7")))

HCOOHtoCHOFH4: {formate} + {ATP} + {CH2FH4, FFH2, FH2f, FH4} -> {CHOFH4}
ATIC7: {CHOFH4} + {AICAR, FGAR} -> {CH2FH4, FFH2, FH2f, FH4}


In [9]:
a, b = gr.getMoleculeLinkage("{CHOFH4}", ["ATIC7"])
print(a)
print(b)

{'CHOFH4'}
[]


In [13]:
a, b = gr.getMoleculeLinkage("{CH2FH4, FFH2, FH2f, FH4}", ["ATIC7", "HCOOHtoCHOFH4"])
print(a)
print(b)

{'FH4'}
[]


In [21]:
echelon_error = m.echelon_errors[0]
echelon_error

MTHFD: {NADP} + {AICAR, FGAR} -> 

In [124]:
result_row = m.echelon_df[echelon_error.label]
nonzero_result_row = result_row[result_row.to_numpy().nonzero()[0]]
nonzero_result_row.index

Index(['{NADP}', '{AICAR, FGAR}'], dtype='object')

In [41]:
if m.rref_df is None:
  operation_df = m.lower_inverse
else:
  operation_df = m.rref_operation.dot(m.lower_inverse)
operation_df
op_row = operation_df.T[echelon_error.label]
nonzero_op_rows = op_row[op_row.to_numpy().nonzero()[0]]
nonzero_op_rows.index

Index(['ATIC7', 'MTHFD'], dtype='object')

In [49]:
for r in nonzero_op_rows.index:
  reaction = simple.getReaction(r)
  print(reaction.makeIdentifier(is_include_kinetics=False))
  print(m.convertReactionToSOMReaction(reaction))
  print()

ATIC7: CHOFH4 + AICAR -> FH4
ATIC7: {CHOFH4} + {AICAR, FGAR} -> {CH2FH4, FFH2, FH2f, FH4}

MTHFD: CH2FH4 + NADP -> CHOFH4
MTHFD: {NADP} + {CH2FH4, FFH2, FH2f, FH4} -> {CHOFH4}



In [55]:
# find out canceled reactants/products first
# meaning, nonzeros that both exist in atic7 and mthfd but not in the result_row
atic7_row = m.som_stoichiometry_matrix[nonzero_op_rows.index[0]]
mthfd_row = m.som_stoichiometry_matrix[nonzero_op_rows.index[1]]
nonzero_atic7_row = atic7_row[atic7_row.to_numpy().nonzero()[0]]
nonzero_mthfd_row = mthfd_row[mthfd_row.to_numpy().nonzero()[0]]
print(nonzero_atic7_row.index)
print(nonzero_mthfd_row.index)

Index(['{CHOFH4}', '{CH2FH4, FFH2, FH2f, FH4}', '{AICAR, FGAR}'], dtype='object')
Index(['{CHOFH4}', '{NADP}', '{CH2FH4, FFH2, FH2f, FH4}'], dtype='object')


In [57]:
nonzero_atic7_row.index.intersection(nonzero_mthfd_row.index)

Index(['{CHOFH4}', '{CH2FH4, FFH2, FH2f, FH4}'], dtype='object')

In [123]:
nonzero_op_rows.index

Index(['ATIC7', 'MTHFD'], dtype='object')

In [63]:
nonzero_elements = m.som_stoichiometry_matrix.index
for r in nonzero_op_rows.index:
  row = m.som_stoichiometry_matrix[r]
  nonzero_elements = nonzero_elements.intersection(row[row.to_numpy().nonzero()[0]].index)
# finally, subtract remaining nonzero elements from nonzero_result_row
canceled_elements = nonzero_elements.difference(nonzero_result_row.index)
print(canceled_elements)

Index(['{CH2FH4, FFH2, FH2f, FH4}', '{CHOFH4}'], dtype='object')


In [73]:
def getSOM(som_name):
  for node in m.nodes:
    if node.identifier == som_name:
      return node

{CH2FH4, FFH2, FH2f, FH4}

In [77]:
som1 = getSOM(canceled_elements[0])
som2 = getSOM(canceled_elements[1])

{CHOFH4}

In [125]:
nonzero_op_rows.index

Index(['ATIC7', 'MTHFD'], dtype='object')

In [121]:
# this is for each som that were eliminated by matrix operation
molecules = {m.name for m in som1.molecules}
linked_molecules = set()
for r in nonzero_op_rows.index:
  reaction = simple.getReaction(r)
  reactants = {m.molecule.name for m in reaction.reactants}
  products = {m.molecule.name for m in reaction.products}
  som_molecules = molecules.intersection(reactants)
  som_molecules = som_molecules.union(molecules.intersection(products))
  linked_molecules = linked_molecules.union(som_molecules)
# now need to path (reactions) between molecules within the linked_molecules
linked_reactions = []
for sr in som1.reactions:
  sreactants = {m.molecule.name for m in sr.reactants}
  sproducts = {m.molecule.name for m in sr.products}
  if sreactants.intersection(linked_molecules) and \
     sproducts.intersection(linked_molecules):
    linked_reactions.append(sr.label)
# can generate report that will create equality between molecules -> maximum amount of information

In [122]:
print(linked_molecules)
print(linked_reactions)

{'CH2FH4', 'FH4'}
['SHMTr']


In [113]:
som1.reactions

{DHFReductase: FH2f -> FH4; cell * kter * FH2b,
 FFH2syn: FH2f -> FFH2; cell * Vm * FH2f,
 SHMTr: CH2FH4 -> FH4; cell * (Vm * (glycine / Km2 / (1 + glycine / Km2)) * (CH2FH4 / Km1) / (1 + CH2FH4 / Km1))}

In [120]:
sreactants

{'FH2f'}